# API for Game Data

In [254]:
import requests
import pandas as pd

def get_nfl_games(year: int) -> pd.DataFrame:

    header = {"User-Agent": "xcs5hg@virginia.edu"}

    def get_json(url: str):
        r = requests.get(url, headers=header)
        r.raise_for_status()
        return r.json()

    season_url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{year}"
    season_data = get_json(season_url)

    season_parts = {}  

    for t in season_data["types"]["items"]:
        t_data = get_json(t["$ref"])
        name = t_data.get("name")
        if name in ["Regular Season", "Postseason"]:
            season_parts[name] = t_data

    if not season_parts:
        raise ValueError(f"No Regular/Postseason types found for {year}")

    event_urls = []

    for part_name, part in season_parts.items():
        weeks_url = part["weeks"]["$ref"]
        weeks_data = get_json(weeks_url)
        week_items = weeks_data.get("items", [])

        for w in week_items:
            week = get_json(w["$ref"])
            events_url = week["events"]["$ref"]
            events_data = get_json(events_url)
            event_urls.extend(e["$ref"] for e in events_data.get("items", []))

    records = []

    for event_url in event_urls:
        event = get_json(event_url)

        game_id = event.get("id")
        date = event.get("date")

        comp = event["competitions"][0]
        competitors = comp.get("competitors", [])

        home_team = None
        away_team = None

        for c in competitors:
            side = c.get("homeAway") 
            team_obj = c.get("team", {})

            if "$ref" in team_obj:
                team_data = get_json(team_obj["$ref"])
            else:
                team_data = team_obj

            team_name = (
                team_data.get("displayName")
                or team_data.get("name")
                or team_data.get("abbreviation")
            )

            if side == "home":
                home_team = team_name
            elif side == "away":
                away_team = team_name

        records.append(
            {
                "game_id": game_id,
                "date": date,
                "home_team": home_team,
                "away_team": away_team,
            }
        )

    return pd.DataFrame(records)

In [ ]:
df_games_2024 = get_nfl_games(2024)


     game_id               date            home_team            away_team
0  401671789  2024-09-06T00:40Z   Kansas City Chiefs     Baltimore Ravens
1  401671805  2024-09-07T00:15Z  Philadelphia Eagles    Green Bay Packers
2  401671744  2024-09-08T17:00Z      Atlanta Falcons  Pittsburgh Steelers
3  401671617  2024-09-08T17:00Z        Buffalo Bills    Arizona Cardinals
4  401671719  2024-09-08T17:00Z        Chicago Bears     Tennessee Titans
286


In [256]:
df_games_2024

,game_id,date,home_team,away_team
0,401671789,2024-09-06T00:40Z,Kansas City Chiefs,Baltimore Ravens
1,401671805,2024-09-07T00:15Z,Philadelphia Eagles,Green Bay Packers
2,401671744,2024-09-08T17:00Z,Atlanta Falcons,Pittsburgh Steelers
3,401671617,2024-09-08T17:00Z,Buffalo Bills,Arizona Cardinals
4,401671719,2024-09-08T17:00Z,Chicago Bears,Tennessee Titans
...,...,...,...,...
281,401671886,2025-01-19T23:30Z,Buffalo Bills,Baltimore Ravens
282,401671888,2025-01-26T20:00Z,Philadelphia Eagles,Washington Commanders
283,401671887,2025-01-26T23:30Z,Kansas City Chiefs,Buffalo Bills
284,401701113,2025-02-02T20:00Z,NFC,AFC


# Odds API (Not exactly what we want)

In [56]:
import requests

header = {"User-Agent": "xcs5hg@virginia.edu"}

def get_game_odds_and_probabilities(game_id: int):

    base = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl"
    
    def get_json(url):
        r = requests.get(url, headers=header)
        r.raise_for_status()
        return r.json()

    event_url = f"{base}/events/{game_id}"
    event = get_json(event_url)

    comp = event["competitions"][0]
    comp_id = comp["id"]


    probs_url = (
        f"{base}/events/{game_id}/competitions/{comp_id}/probabilities?limit=500"
    )
    probabilities = get_json(probs_url)

    odds_url = (
        f"{base}/events/{game_id}/competitions/{comp_id}/odds"
    )
    odds = get_json(odds_url)

    return {
        "game_id": game_id,
        "competition_id": comp_id,
        "event": event,
        "probabilities": probabilities,
        "odds": odds,
    }

In [59]:
odds = get_game_odds_and_probabilities(401671789)

In [71]:
pd.json_normalize(odds['odds']['items'])

,$ref,details,overUnder,spread,overOdds,underOdds,links,moneylineWinner,spreadWinner,provider.$ref,...,bettingOdds.teamOdds.preMatchSpreadAway.betSlipUrl,bettingOdds.teamOdds.preMatchTotalUnder.oddId,bettingOdds.teamOdds.preMatchTotalUnder.value,bettingOdds.teamOdds.preMatchTotalUnder.betSlipUrl,bettingOdds.teamOdds.preMatchTotalHandicap.oddId,bettingOdds.teamOdds.preMatchTotalHandicap.value,bettingOdds.teamOdds.preMatchTotalHandicap.betSlipUrl,bettingOdds.teamOdds.preMatchSpreadHandicapHome.oddId,bettingOdds.teamOdds.preMatchSpreadHandicapHome.value,bettingOdds.teamOdds.preMatchSpreadHandicapHome.betSlipUrl
0,http://sports.core.api.espn.com/v2/sports/foot...,KC -2.5,45.5,-2.5,-115.0,-105.0,"[{'language': 'en-US', 'rel': ['home', 'deskto...",False,False,http://sports.core.api.espn.com/v2/sports/foot...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://sports.core.api.espn.com/v2/sports/foot...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,http://sports.core.api.espn.com/v2/sports/foot...,...,https://www.bet365.com/dl/sportsbookredirect?a...,484455341,10/11,https://www.bet365.com/dl/sportsbookredirect?a...,484455341,46.5,https://www.bet365.com/dl/sportsbookredirect?a...,484455319,-2.5,https://www.bet365.com/dl/sportsbookredirect?a...
2,http://sports.core.api.espn.com/v2/sports/foot...,KC -10.5,48.5,-10.5,185.0,-275.0,"[{'language': 'en-US', 'rel': ['home', 'deskto...",False,False,http://sports.core.api.espn.com/v2/sports/foot...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Athletes API

In [332]:
import requests
import pandas as pd

def get_team_roster(team_id: int) -> pd.DataFrame:
    headers = {"User-Agent": "xcs5hg@virginia.edu"}

    meta_url = f"https://site.api.espn.com/apis/site/v2/sports/football/nfl/teams/{team_id}"
    meta = requests.get(meta_url, headers=headers).json()
    team_name = meta.get("team", {}).get("displayName")

    roster_url = f"https://site.api.espn.com/apis/site/v2/sports/football/nfl/teams/{team_id}/roster"
    data = requests.get(roster_url, headers=headers).json()

    groups = data.get("athletes", [])
    players = []

    for group in groups:
        for p in group.get("items", []):

            pos_info = p.get("position", {})
            real_pos_name = pos_info.get("name")
            real_pos_abbr = pos_info.get("abbreviation")

            players.append({
                "athlete_id": p.get("id"),
                "full_name": p.get("fullName"),
                "display_name": p.get("displayName"),

                # true football position
                "position_name": real_pos_name,      
                "position_abbr": real_pos_abbr,      

                "jersey": p.get("jersey"),
                "age": p.get("age"),
                "height": p.get("displayHeight"),
                "weight": p.get("displayWeight"),
                "team_id": team_id,
                "team_name": team_name
            })

    return pd.DataFrame(players)

In [ ]:
df = get_team_roster(20, 2024)
df.head()
len(df)

In [339]:
import requests
import pandas as pd

def get_team_metadata(team_id: int, year: int = 2024) -> dict:

    headers = {"User-Agent": "xcs5hg@virginia.edu"}

    meta_url = (
        f"https://site.api.espn.com/apis/site/v2/sports/football/nfl/teams/{team_id}"
        f"?season={year}"
    )

    r = requests.get(meta_url, headers=headers)
    r.raise_for_status()

    data = r.json()
    team = data.get("team", {})

    out = {
        "team_id": team_id,
        "season": year,
        "team_uid": team.get("uid"),
        "display_name": team.get("displayName"),
        "short_name": team.get("shortDisplayName"),
        "name": team.get("name"),
        "abbreviation": team.get("abbreviation"),
        "location": team.get("location"),
        "nickname": team.get("nickname"),
        "color": team.get("color"),
        "alternate_color": team.get("alternateColor"),
        "logo": team.get("logos", [{}])[0].get("href") if team.get("logos") else None,
        "venue": team.get("venue", {}).get("fullName"),
    }

    return out

In [341]:
team_meta = get_team_metadata(12, 2023)   # Kansas City Chiefs, 2024 season
team_meta

{'team_id': 12,
 'season': 2023,
 'team_uid': 's:20~l:28~t:12',
 'display_name': 'Kansas City Chiefs',
 'short_name': 'Chiefs',
 'name': 'Chiefs',
 'abbreviation': 'KC',
 'location': 'Kansas City',
 'nickname': 'Chiefs',
 'color': 'e31837',
 'alternate_color': 'ffb612',
 'logo': 'https://a.espncdn.com/i/teamlogos/nfl/500/kc.png',
 'venue': None}

In [333]:
team_ids = [
    22,  # Arizona Cardinals
    1,   # Atlanta Falcons
    33,  # Baltimore Ravens
    2,   # Buffalo Bills
    29,  # Carolina Panthers
    3,   # Chicago Bears
    4,   # Cincinnati Bengals
    5,   # Cleveland Browns
    6,   # Dallas Cowboys
    7,   # Denver Broncos
    8,   # Detroit Lions
    9,   # Green Bay Packers
    34,  # Houston Texans
    11,  # Indianapolis Colts
    30,  # Jacksonville Jaguars
    12,  # Kansas City Chiefs
    13,  # Las Vegas Raiders
    24,  # Los Angeles Chargers
    14,  # Los Angeles Rams
    15,  # Miami Dolphins
    16,  # Minnesota Vikings
    17,  # New England Patriots
    18,  # New Orleans Saints
    19,  # New York Giants
    20,  # New York Jets
    21,  # Philadelphia Eagles
    23,  # Pittsburgh Steelers
    25,  # San Francisco 49ers
    26,  # Seattle Seahawks
    27,  # Tampa Bay Buccaneers
    10,  # Tennessee Titans
    28   # Washington Commanders
]

In [336]:
df = get_team_roster(20, 2024)


In [338]:
df

""


# Play by play API 

In [52]:
import requests

def get_game_plays(game_id: int):

    header = {"User-Agent": "xcs5hg@virginia.edu"}

    event_url = (
        f"https://sports.core.api.espn.com/v2/"
        f"sports/football/leagues/nfl/events/{game_id}"
    )
    event = requests.get(event_url, headers=header).json()

    competition = event["competitions"][0]
    comp_id = competition["id"]

    plays_url = (
        f"https://sports.core.api.espn.com/v2/"
        f"sports/football/leagues/nfl/events/{game_id}/competitions/{comp_id}/plays?limit=500"
    )

    plays = requests.get(plays_url, headers=header).json()
    
    record = []
    
    for item in plays["items"]:
        record.append(item)
        
    keep_cols = [
    "text",
    "period.number",
    "clock.value",
    "start.down",
    "start.distance",
    "start.yardLine",
    "start.yardsToEndzone",
    "end.yardsToEndzone",
    "homeScore",
    "awayScore",
    "statYardage",
    "scoringPlay",
    "scoreValue",
    "type.text",
    "type.abbreviation"]
    
    data = pd.json_normalize(record)
    data = data[keep_cols]  
    
    
    return data

In [53]:
plays = get_game_plays(401671789)
plays


,text,period.number,clock.value,start.down,start.distance,start.yardLine,start.yardsToEndzone,end.yardsToEndzone,homeScore,awayScore,statYardage,scoringPlay,scoreValue,type.text,type.abbreviation
0,GAME,0,900.0,0,0,0,0,65,0,0,0,False,0,Coin Toss,NaN
1,H.Butker kicks 65 yards from KC 35 to end zone...,1,900.0,0,0,35,65,70,0,0,0,False,0,Kickoff,K
2,(Shotgun) D.Henry left end to BLT 32 for 2 yar...,1,900.0,1,10,70,70,68,0,0,2,False,0,Rush,RUSH
3,(Shotgun) L.Jackson pass short right to Z.Flow...,1,859.0,2,8,68,68,73,0,0,-5,False,0,Pass Reception,REC
4,(Shotgun) L.Jackson pass short right to J.Hill...,1,835.0,2,13,73,73,71,0,0,2,False,0,Pass Reception,REC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,"(No Huddle, Shotgun) L.Jackson pass incomplete...",4,18.0,1,10,10,10,10,27,20,0,False,0,Pass Incompletion,NaN
185,Timeout #4 by BLT at 00:10.,4,10.0,2,10,10,10,10,27,20,0,False,0,Timeout,TO
186,(Shotgun) L.Jackson pass incomplete short midd...,4,10.0,2,10,10,10,10,27,20,0,False,0,Pass Incompletion,NaN
187,(Shotgun) L.Jackson pass short middle to I.Lik...,4,5.0,3,10,10,10,10,27,20,0,False,0,Pass Incompletion,NaN


In [54]:
len(plays)

189